In [44]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_excel('NLP_input_20211008.xlsx')

In [3]:
df.head()

,Taxpayer Number,Ori_Tax_Name,Location Address,Ori_AggName,Location City,Location Zip,Location State,Yelp_ID,Input?,Yelp_Name,Yelp_Address
0,10106117038,KOBE JAPANESE STEAKHOUSE,13492 N HIGHWAY 183 STE 380,KOBE JAPANESE STEAKHOUSE13492 N HIGHWAY 183 ST...,AUSTIN,78750,TX,"KOBE Japanese Steakhouse13492 Research Blvd, S...",1,KOBE Japanese Steakhouse,"13492 Research Blvd, Ste 380"
1,10108780338,TINIEST BAR IN TEXAS,817 W 5TH ST,TINIEST BAR IN TEXAS817 W 5TH ST,AUSTIN,78703,TX,zKccBxZ5pE8Qbi1lnZTOaQ,1,Tiniest Bar in Texas,817 W 5th St
2,10207558767,WATERLOO ICE HOUSE,9600 ESCARPMENT BLVD STE 800,WATERLOO ICE HOUSE9600 ESCARPMENT BLVD STE 800,AUSTIN,78749,TX,SW6ECNwTIk8wzsMxGNFdcw,1,Waterloo Ice House Escarpment,9600 Escarpment Blvd
3,10304776353,FUJIYAMA JAPANESE STEAKHOUSE,4815 W BRAKER LN STE 580,FUJIYAMA JAPANESE STEAKHOUSE4815 W BRAKER LN S...,AUSTIN,78759,TX,H3NT5eQaIm5vMpqJr4d1RA,1,Fujiyama Japanese Steakhouse,"4815 W Braker Ln, Ste 580"
4,10436052947,NUTTY BROWN CAFE,12225 W HIGHWAY 290,NUTTY BROWN CAFE12225 W HIGHWAY 290,AUSTIN,78737,TX,60WzJUIw-K9EYutAsF3U7g,1,Nutty Brown Cafe,12225 Hwy 290 W


In [4]:
df.shape

(909, 11)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Taxpayer Number   909 non-null    int64 
 1   Ori_Tax_Name      909 non-null    object
 2   Location Address  909 non-null    object
 3   Ori_AggName       909 non-null    object
 4   Location City     909 non-null    object
 5   Location Zip      909 non-null    int64 
 6   Location State    909 non-null    object
 7   Yelp_ID           909 non-null    object
 8   Input?            909 non-null    int64 
 9   Yelp_Name         909 non-null    object
 10  Yelp_Address      909 non-null    object
dtypes: int64(3), object(8)
memory usage: 78.2+ KB


In [6]:
from fuzzywuzzy import fuzz

In [7]:
def get_ratio(row):
    loc_add = row['Location Address']
    yelp_add = row['Yelp_Address']
    return fuzz.token_set_ratio(loc_add, yelp_add)
len(df[df.apply(get_ratio, axis=1) > 85]) / len(df)

0.9438943894389439

#### 94% of the text have matching score of 85 or above 

In [8]:
## fuzzywuzzy matching score 
df['score'] = df.apply(get_ratio, axis=1)

In [10]:
# Check the matching score at the end of the dataframe columns
df.head()

,Taxpayer Number,Ori_Tax_Name,Location Address,Ori_AggName,Location City,Location Zip,Location State,Yelp_ID,Input?,Yelp_Name,Yelp_Address,score
0,10106117038,KOBE JAPANESE STEAKHOUSE,13492 N HIGHWAY 183 STE 380,KOBE JAPANESE STEAKHOUSE13492 N HIGHWAY 183 ST...,AUSTIN,78750,TX,"KOBE Japanese Steakhouse13492 Research Blvd, S...",1,KOBE Japanese Steakhouse,"13492 Research Blvd, Ste 380",65
1,10108780338,TINIEST BAR IN TEXAS,817 W 5TH ST,TINIEST BAR IN TEXAS817 W 5TH ST,AUSTIN,78703,TX,zKccBxZ5pE8Qbi1lnZTOaQ,1,Tiniest Bar in Texas,817 W 5th St,100
2,10207558767,WATERLOO ICE HOUSE,9600 ESCARPMENT BLVD STE 800,WATERLOO ICE HOUSE9600 ESCARPMENT BLVD STE 800,AUSTIN,78749,TX,SW6ECNwTIk8wzsMxGNFdcw,1,Waterloo Ice House Escarpment,9600 Escarpment Blvd,100
3,10304776353,FUJIYAMA JAPANESE STEAKHOUSE,4815 W BRAKER LN STE 580,FUJIYAMA JAPANESE STEAKHOUSE4815 W BRAKER LN S...,AUSTIN,78759,TX,H3NT5eQaIm5vMpqJr4d1RA,1,Fujiyama Japanese Steakhouse,"4815 W Braker Ln, Ste 580",100
4,10436052947,NUTTY BROWN CAFE,12225 W HIGHWAY 290,NUTTY BROWN CAFE12225 W HIGHWAY 290,AUSTIN,78737,TX,60WzJUIw-K9EYutAsF3U7g,1,Nutty Brown Cafe,12225 Hwy 290 W,88


### Using k-nearest neighbour algorithm - Implementation

In [32]:
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
import time
pd.set_option('display.max_colwidth', -1)

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_16700/3280276915.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [33]:
import re
def cleaning(string, n=3):
    string = string.lower()
    string = string.encode("ascii", errors="ignore").decode()
    string = string.replace('&', 'and')
    string = string.replace('-', ' ').replace(',',' ')
    string = string.title()
    string = re.sub(' +',' ',string).strip()
    string = ' ' + string + ' '
    string = re.sub(r'[,-./]|\sBD', r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]    

In [17]:
loc_add = df['Location Address'].tolist()
yelp_add = df['Yelp_Address'].tolist()

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

def tfidf_matching(list1, list2):
    vectorizing = TfidfVectorizer(analyzer=cleaning, lowercase=False)
    tfidf = vectorizing.fit_transform(list2)
    nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
    distances, indices = nbrs.kneighbors(vectorizing.transform(list1))
    matches = [(round(distances[i][0], 2), list1[i], list2[j[0]]) 
               for i, j in enumerate(indices)]
    matches = pd.DataFrame(matches, 
                           columns=['score', 'original', 'matched'])
    return matches

In [49]:
tfidf_matching(loc_add, yelp_add).head(1)
# higher the score more the distance between vectors, lesser score value denotes the closeness of match

,score,original,matched
0,1.04,13492 N HIGHWAY 183 STE 380,"13492 Research Blvd, Ste 380"


In [46]:
matched_df = tfidf_matching(loc_add, yelp_add)

In [47]:
matched_df.head()

,score,original,matched
0,1.04,13492 N HIGHWAY 183 STE 380,"13492 Research Blvd, Ste 380"
1,0.00,817 W 5TH ST,817 W 5th St
2,0.44,9600 ESCARPMENT BLVD STE 800,9600 Escarpment Blvd
3,0.00,4815 W BRAKER LN STE 580,"4815 W Braker Ln, Ste 580"
4,0.79,12225 W HIGHWAY 290,4809 W Highway 290


In [48]:
matched_df.sort_values(by=['score'])
# higher the score more the distance between vectors, lesser score value denotes the closeness of match
# The matches score with value 0.00 denotes exact or near exact match

,score,original,matched
454,0.00,3225 AMY DONOVAN PLZ,3225 Amy Donovan Plz
513,0.00,2101 MONTOPOLIS DR UNIT 23,"2101 Montopolis Dr, Unit 23"
514,0.00,101 LAKEWAY DR,101 Lakeway Dr
515,0.00,1500 S LAMAR BLVD STE 150,"1500 S Lamar Blvd, Ste 150"
516,0.00,2207 MANOR RD,2207 Manor Rd
517,0.00,215 E 6TH ST,215 E 6th St
518,0.00,1100 S LAMAR BLVD STE 1150,"1100 S Lamar Blvd, Ste 1150"
520,0.00,2700 S LAMAR BLVD,2700 S Lamar Blvd
521,0.00,1201 S LAMAR BLVD,1201 S Lamar Blvd
523,0.00,1502 W BEN WHITE BLVD,1502 W Ben White Blvd
